In [19]:
# ========================
# 1. CREACIÓN DISPONIBILIDAD DOCENTE SIMULADA
# ========================

import pandas as pd
import numpy as np
import random

# Cargar archivos necesarios
docentes_minimos = pd.read_csv("..\\Data\\Extraidos\\docentes_minimos.csv", sep=";", encoding="utf-8-sig")
duracion_asignatura = pd.read_csv("..\\Data\\Extraidos\\duracion_asignatura.csv", sep=";", encoding="utf-8-sig")

# Crear diccionarios para duración y ciclo
ciclo_dict = duracion_asignatura.drop_duplicates(subset="CodAsignatura").set_index("CodAsignatura")["Ciclo"].to_dict()

# Inicializar parámetros
np.random.seed(32)
contador_docente = 0
filas_disponibilidad = []
excepciones = 3  # generar 3 horarios sin reglas

for _, row in docentes_minimos.iterrows():
    cod_asig = row["CodAsignatura"]
    duracion = float(row["DuracionHoras"])
    ciclo = int(ciclo_dict.get(cod_asig, 1))  # default ciclo 1 si no se encuentra

    # Validar cantidad mínima de docentes
    try:
        min_docentes = int(row["NumGrupos"])
    except:
        min_docentes = 1

    if min_docentes < 1:
        min_docentes = 1

    for _ in range(min_docentes):
        cod_docente = f"Z{str(contador_docente).zfill(6)}"
        contador_docente += 1

        # Reglas según ciclo
        if excepciones > 0:
            posibles_dias = ["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO"]
            horas_validas = pd.date_range("08:00", "21:00", freq="1h").strftime("%H:%M").tolist()
            excepciones -= 1
        elif ciclo >= 6:
            dia = np.random.choice(["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO"])
            if dia == "SABADO":
                horas_validas = pd.date_range("08:00", "21:00", freq="1h").strftime("%H:%M").tolist()
            else:
                horas_validas = pd.date_range("18:00", "21:00", freq="1h").strftime("%H:%M").tolist()
        else:
            dia = np.random.choice(["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES"])
            horas_validas = pd.date_range("08:00", "12:00", freq="1h").strftime("%H:%M").tolist()
            horas_validas += pd.date_range("14:00", "17:00", freq="1h").strftime("%H:%M").tolist()

        # Validar que haya suficiente espacio para la duración
        horas_inicio = [h for h in horas_validas if pd.to_datetime(h) + pd.Timedelta(hours=duracion) <= pd.to_datetime("22:00")]
        if not horas_inicio:
            continue

        hi = np.random.choice(horas_inicio)
        hf = (pd.to_datetime(hi) + pd.Timedelta(hours=duracion)).strftime("%H:%M")
        if 'dia' not in locals():
            dia = np.random.choice(["LUNES", "MARTES", "MIERCOLES", "JUEVES", "VIERNES", "SABADO"])

        filas_disponibilidad.append({
            "CodDocente": cod_docente,
            "CodAsignatura": cod_asig,
            "Día": dia,
            "HoraInicio": hi,
            "HoraFin": hf,
            "DuracionHoras": duracion,
            "Ciclo": ciclo
        })

# Convertir a DataFrame
df_result = pd.DataFrame(filas_disponibilidad)

# Exportar
df_result.to_csv("..\\Data\\Extraidos\\disponibilidad_docentes_simulada.csv", index=False, encoding="utf-8-sig")
print(f"Total de docentes generados: {df_result['CodDocente'].nunique()}")
print(df_result.head())


Total de docentes generados: 73
  CodDocente CodAsignatura      Día HoraInicio HoraFin  DuracionHoras  Ciclo
0    Z000000      202W0301  VIERNES      15:00   20:00            5.0      3
1    Z000001      202W0301  VIERNES      19:00   22:00            3.0      3
2    Z000002      202W0302  VIERNES      13:00   18:00            5.0      3
3    Z000003      202W0303  VIERNES      17:00   21:00            4.0      3
4    Z000004      202W0304   JUEVES      16:00   20:00            4.0      3
